In [1]:
import warnings, sys, os, gc
from os.path import join
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# import torch; print(torch.cuda.is_available())

import libemg
from libemg.datasets import get_dataset_list
from libemg.feature_extractor import FeatureExtractor

import numpy as np, pandas as pd
import random, copy, time, csv
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

from utils import *
from Models.CNN import CNN

SEED = 13; random.seed(SEED); np.random.seed(SEED)
GENERATOR = torch.manual_seed(SEED)
MMAP_MODE = 'r'

Bluepy not installed...


In [ ]:
from SPACESHIP import SPACESHIP
dataset = SPACESHIP()
data = dataset.prepare_data()

path = join(PATH, 'ssl')
os.makedirs(path, exist_ok=True)

np.save(join(path, 'ssl_data'), data)

ssl_windows, ssl_meta = data.parse_windows(SEQ, SSL_INC)

np.save(join(path, 'ssl_windows'), ssl_windows.astype(DTYPE))
np.save(join(path, 'ssl_meta'), ssl_meta)

ssl_windows.shape

In [ ]:
from gForceSGT import gForceSGT
dataset = gForceSGT()
data = dataset.prepare_data()

path = join(PATH, 'sgt')
os.makedirs(path, exist_ok=True)

np.save(join(path, 'sgt_data'), data)

In [ ]:
path = join(PATH, 'sgt')
sgt_data = np.load(join(path, 'sgt_data.npy'), allow_pickle=True).item()
_data = sgt_data.isolate_data("subjects", list(range(1)), fast=True)

train_data = _data.isolate_data("rep_forms", [0], fast=True)
train_data = _data.isolate_data("reps", list(range(1)), fast=True)
train_windows, train_meta = train_data.parse_windows(SEQ, INC)
np.min(train_windows), np.mean(train_windows), np.max(train_windows)

In [2]:
path = join(PATH, 'ssl')
train_windows = np.load(join(path, 'ssl_windows.npy'), mmap_mode=MMAP_MODE)

In [3]:
a = train_windows[::10]

In [8]:
ag = augment(torch.from_numpy(a))

In [10]:
f = FeatureExtractor()
af = f.extract_features(['WENG'], a, feature_dic={'WENG_fs': 1000}, array=True)
np.min(af), np.mean(af), np.max(af)

(-23.01625, -17.154947, -4.8020205)

In [9]:
f = FeatureExtractor()
af = f.extract_features(['WENG'], ag, feature_dic={'WENG_fs': 1000}, array=True)
np.min(af), np.mean(af), np.max(af)

(-23.016283, -17.165705, -4.612915)

In [ ]:
af

In [ ]:
a = extract_features(train_windows, ['WENG'], feature_dic={'WENG_fs': 1000})

In [ ]:
a.shape

In [ ]:
train_windows.shape

In [ ]:
from gForceSGT import gForceSGT
dataset = gForceSGT()
data = dataset.prepare_data()

train_data = data.isolate_data("subjects", list(range(4)), fast=True)
val_data = data.isolate_data("subjects", list(range(4, 5)), fast=True)
test_data = data.isolate_data("subjects", list(range(6)), fast=True)

path = join(PATH, 'sgt')
os.makedirs(path, exist_ok=True)

np.save(join(path, 'train_data'), train_data)
np.save(join(path, 'val_data'), val_data)
np.save(join(path, 'test_data'), test_data)

train_windows, train_meta = train_data.parse_windows(SEQ, INC)
val_windows, val_meta = val_data.parse_windows(SEQ, INC)
test_windows, test_meta = test_data.parse_windows(SEQ, INC)

np.save(join(path, 'train_windows'), train_windows.astype(DTYPE))
np.save(join(path, 'train_meta'), train_meta)

np.save(join(path, 'val_windows'), val_windows.astype(DTYPE))
np.save(join(path, 'val_meta'), val_meta)

np.save(join(path, 'test_windows'), test_windows.astype(DTYPE))
np.save(join(path, 'test_meta'), test_meta)

train_windows.shape, val_windows.shape, test_windows.shape

In [ ]:
path = join(PATH, 'ssl')
ssl_windows = np.load(join(path, 'ssl_windows.npy'), mmap_mode=MMAP_MODE)

path = join(PATH, 'sgt')
train_windows = np.load(join(path, 'train_windows.npy'), mmap_mode=MMAP_MODE)
train_meta = np.load(join(path, 'train_meta.npy'), allow_pickle=True).item()
val_windows = np.load(join(path, 'val_windows.npy'), mmap_mode=MMAP_MODE)
val_meta = np.load(join(path, 'val_meta.npy'), allow_pickle=True).item()
test_windows = np.load(join(path, 'test_windows.npy'), mmap_mode=MMAP_MODE)
test_meta = np.load(join(path, 'test_meta.npy'), allow_pickle=True).item()